<a href="https://colab.research.google.com/github/TurkuNLP/textual-data-analysis-course/blob/main/squad_as_sent_classification_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QA as sentence classification

## Task 1:

* Familiarize yourself with the whole notebook and what is going on there
* At the very bottom there is a place where you can simply run a trained model
* Print some interesting correct and incorrect classifications (e.g. cases where the model failed to answer "yes" - false negatives, or cases where it overpredicted a "yes" - false positives)
* Do these make sense? Would you do better?

## Task 2:

* Try to see if you can run the training yourself on the dataset
* AND add precision and recall to the reported metrics so you can better see what happens to the positive class
* Make sure you save the models etc

## Task 3:

* You will notice that the basic model is heavily skewed towards predicting the (more common) negative class
* The correct way to address this is by introducing class weights into training
* Try, for example, to train a model with a 0.9 weight on the positive class and 0.1 on the negative, does anything change? And what if you turn the weights around, and give the positive class a 0.1 weight, will the model ever predict any?
* Google will help you with how to add class weights into the training process (it involves overriding the compute_loss() method of the trainer)

## Task 4:

* Those of you who want, once you make it to the point you can train your own model, you can do it on Finnish
* The Finnish QA data is pointed to below

In [ ]:
!pip3 install evaluate transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import transformers
import datasets
import evaluate

# SQuAD v2

* First, we will load and explore the SQuAD v2 dataset
* It is also quite important to read through the SQuAD v2 paper to understand the finer points of the data
* `squad_v2` is the original English dataset
* `TurkuNLP/squad_v2_fi` is a machine-translated version produced by TurkuNLP

In [ ]:
#squad_dataset_full=datasets.load_dataset("TurkuNLP/squad_v2_fi")
squad_dataset_full=datasets.load_dataset("squad_v2")
squad_dataset_full=squad_dataset_full.shuffle()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
for item in squad_dataset_full["train"].select(range(5)):
    print(item)
    print()


{'id': '573057502461fd1900a9cd7a', 'title': 'Swaziland', 'context': 'The Reed Dance today is not an ancient ceremony but a development of the old "umchwasho" custom. In "umchwasho", all young girls were placed in a female age-regiment. If any girl became pregnant outside of marriage, her family paid a fine of one cow to the local chief. After a number of years, when the girls had reached a marriageable age, they would perform labour service for the Queen Mother, ending with dancing and feasting. The country was under the chastity rite of "umchwasho" until 19 August 2005.', 'question': 'From where did the Reed Dance originate?', 'answers': {'text': ['the old "umchwasho" custom'], 'answer_start': [69]}}

{'id': '5ad31566604f3c001a3fdb44', 'title': 'Cardinal_(Catholicism)', 'context': "When in choir dress, a Latin-rite cardinal wears scarlet garments — the blood-like red symbolizes a cardinal's willingness to die for his faith. Excluding the rochet — which is always white — the scarlet ga

# QA as a classification task

* As a warm-up problem, let us cast QA as a classification problem
* The task is: does the string at hand contain the answer to the question?
* This is not how you would do QA but it is a good start, and an interesting problem in its own right
* Maybe we can give ourselves some slack, and go sentence-by-sentence
* So, we need to split the context into sentences, and then assemble a yes/no classification data
* But how do we split the text into sentences?
* Udpipe is an old, but trusty library to do this
* Trained models: https://ufal.mff.cuni.cz/udpipe/1/models#universal_dependencies_25_models
* Go familiarize yourself with that tool and models!
* For Finnish, make sure you use the TDT model (Made in Turku), which is the best
* For English, the EWT (English Web Treebank) model is a good choice

In [ ]:
!pip3 install ufal.udpipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.5/848.5 KB 39.1 MB/s eta 0:00:00


In [ ]:
!wget -O english-ewt.udpipe 'https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/english-ewt-ud-2.5-191206.udpipe?sequence=17&isAllowed=y'

--2023-01-31 18:25:37--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/english-ewt-ud-2.5-191206.udpipe?sequence=17&isAllowed=y
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16309608 (16M) [application/octet-stream]
Saving to: ‘english-ewt.udpipe’

english-ewt.udpipe  100%[===================>]  15.55M  6.60MB/s    in 2.4s    

2023-01-31 18:25:42 (6.60 MB/s) - ‘english-ewt.udpipe’ saved [16309608/16309608]



# Running UDPipe

* This is surprisingly easy, once you figure it out
* The documentation (from which I figured it out) is here:
  * https://ufal.mff.cuni.cz/udpipe/1/users-manual#run_udpipe_tokenizer
  * https://ufal.mff.cuni.cz/udpipe/1/api-reference#pipeline

In [ ]:
import ufal.udpipe
udpipemodel=ufal.udpipe.Model.load("english-ewt.udpipe")
tokenizer=ufal.udpipe.Pipeline(udpipemodel,"tokenizer=ranges","none","none","conllu")

In [ ]:
print(tokenizer.process("I have a dog. The dog is cute. And brown at that."))

# newdoc
# newpar
# sent_id = 1
# text = I have a dog.
1	I	_	_	_	_	_	_	_	TokenRange=0:1
2	have	_	_	_	_	_	_	_	TokenRange=2:6
3	a	_	_	_	_	_	_	_	TokenRange=7:8
4	dog	_	_	_	_	_	_	_	SpaceAfter=No|TokenRange=9:12
5	.	_	_	_	_	_	_	_	TokenRange=12:13

# sent_id = 2
# text = The dog is cute.
1	The	_	_	_	_	_	_	_	TokenRange=14:17
2	dog	_	_	_	_	_	_	_	TokenRange=18:21
3	is	_	_	_	_	_	_	_	TokenRange=22:24
4	cute	_	_	_	_	_	_	_	SpaceAfter=No|TokenRange=25:29
5	.	_	_	_	_	_	_	_	TokenRange=29:30

# sent_id = 3
# text = And brown at that.
1	And	_	_	_	_	_	_	_	TokenRange=31:34
2	brown	_	_	_	_	_	_	_	TokenRange=35:40
3	at	_	_	_	_	_	_	_	TokenRange=41:43
4	that	_	_	_	_	_	_	_	SpaceAfter=No|TokenRange=44:48
5	.	_	_	_	_	_	_	_	SpacesAfter=\n|TokenRange=48:49




# Find the sentence with the answer
* Let's take a shortcut and simply use the `text=` field to find sentences which have the answer
* Of course this might lead to spurious hits where the string is repeated in several sentences, but acts as the correct answer only in one
* The right solution would be to use the `TokenRange` data to find the correct sentence that has the right answer
* I leave that to you as an extra exercise if you wish
* All you need is to parse the output of UDPipe, takes an extra for-loop, really

In [ ]:
import tqdm
def get_sentences(parsed):
    # gather the text= lines with the sentences
    sents=[line.replace("# text = ","") for line in parsed.split("\n") if line.startswith("# text = ")]
    return sents

def qa_to_binary(example,tokenizer):
    context=example["context"]
    question=example["question"]
    sentences=get_sentences(tokenizer.process(context))
    result=[]
    #compare every sentence with every answer, when you find an answer, you have an example
    for sent in sentences:
        for answer in example["answers"]["text"]:
            if answer in sent:
                result.append({"question":question,"context":sent,"label":1,"answer":answer})
                break
        else: #we found no answers, so this sentence is then 0
            result.append({"question":question,"context":sent,"label":0,"answer":None})
    return result

train_processed=[]
for item in tqdm.tqdm(squad_dataset_full["train"].select(range(50000))):
    train_processed.extend(qa_to_binary(item,tokenizer))
test_processed=[]
for item in tqdm.tqdm(squad_dataset_full["validation"].select(range(2000))):
    test_processed.extend(qa_to_binary(item,tokenizer))

                      

        

# Save the data

* Now the usual trick of saving the data in a nice, processed form as jsonl
* And then load as a HF Dataset

In [ ]:
import json
with open("squad_2_binarized_train.json","wt") as f:
    json.dump(train_processed,f,ensure_ascii=False,indent=2)
with open("squad_2_binarized_test.json","wt") as f:
    json.dump(test_processed,f,ensure_ascii=False,indent=2)

In [ ]:
#dset=datasets.load_dataset("json",data_files={"train":"squad_2_binarized_train.json","test":"squad_2_binarized_test.json"},download_mode="force_redownload")
dset=datasets.load_dataset("json",data_files={"train":"http://dl.turkunlp.org/TKO_8964_2023/squad_2_binarized_train.json","test":"http://dl.turkunlp.org/TKO_8964_2023/squad_2_binarized_test.json"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b2a88c8518fe60b6/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dset=dset.shuffle()

# Train the classifier



In [ ]:
MODEL = 'bert-base-cased'
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL)

* The tokenizer now needs a pair of texts (the question and the context)
* For BERT, it will actually then do the right thing with token type IDs
* The two texts are passed as `text` and `text_pair` (for some confusing naming)

In [ ]:
def tokenize(example):
    return tokenizer(text=example["question"],text_pair=example["context"], truncation=True)

dataset = dset.map(tokenize)

  0%|          | 0/259266 [00:00<?, ?ex/s]

  0%|          | 0/10703 [00:00<?, ?ex/s]

In [ ]:
trainer_args = transformers.TrainingArguments(
    output_dir='checkpoints',
    evaluation_strategy='steps',
    logging_strategy='steps',
    load_best_model_at_end=True,
    eval_steps=250,
    logging_steps=250,
    learning_rate=0.000015,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    max_steps=5000,
    save_steps=250,
    save_total_limit=6 #I will have an early stopping with patience 5, so max 6 checkpoints sounds reasonable
)


# How to evaluate?

* The data is imbalanced
* The positive class is much rarer than the negative class
* High accuracy does not tell the whole story
* It will be good to report also precision and recall, so we can gauge what happens to the class

In [ ]:
accuracy = evaluate.load('accuracy')
precision = evaluate.load('precision')
recall = evaluate.load('recall')


def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = outputs.argmax(axis=-1) # pick index of "winning" label
    acc=accuracy.compute(predictions=predictions, references=labels)
    prec=precision.compute(predictions=predictions, references=labels)
    rec=recall.compute(predictions=predictions, references=labels)
    return dict(**acc,**prec,**rec)

* Let's also add EarlyStopping

In [ ]:
estopping_cback=transformers.EarlyStoppingCallback(5)

# Class imbalance

* One way to deal with imbalanced data is to give the minority class higher weight during training
* I.e. the loss is weighted by class-dependent weights
* A way to implement this can be copied straight from the HF documentation
* You can invent some weights, or maybe you can use the formula/code from e.g. here https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html


In [ ]:
import torch

class WeightTrainer(transformers.Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.9],device=logits.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
# A little housekeeping
# to help with various memory leaks

import gc
try:
    del trainer
except:
    pass
try:
    del model
except:
    pass
gc.collect()
torch.cuda.empty_cache()


In [ ]:
#This is something I needed on CSC's puhti
#why, I have no idea
try:
    import mlflow
    mlflow.end_run()
    mlflow.start_run()
except:
    pass

model = transformers.AutoModelForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=2
 )

trainer = WeightTrainer(
    model=model,
    args=trainer_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_accuracy,
    tokenizer=tokenizer,
    callbacks=[estopping_cback], 
)

trainer.train()

In [ ]:
trainer.save_model("english-binarized-weighted.model")

# Trained models

* I left the trained models for you here, in case you want them
* http://dl.turkunlp.org/TKO_8964_2023/

# Test the model

In [ ]:
model=transformers.AutoModelForSequenceClassification.from_pretrained("english-binarized-weighted.model")
tokenizer=transformers.AutoTokenizer.from_pretrained("english-binarized-weighted.model",truncation=True,padding=True)

In [ ]:
dset=datasets.load_from_disk("shuffled_binarized_squad2_dataset")

In [ ]:
pipe=transformers.TextClassificationPipeline(model=model,tokenizer=tokenizer,device=0)

In [ ]:
# Hmm, this is not the right way to go about it, perhaps, but the KeyPairDataset
# I could not get working, even though that would be the right way to do it
# {"text":[listoftexts],"text_pair":[listoftexts]} should have worked according to docs,
# but did not in reality :D
res=pipe([{"text":e["question"],"text_pair":e["context"]} for e in dset["test"].select(range(100))])

In [ ]:
# Let's print out some interesting misclassifications

for prediction,datapoint in zip(res,dset["test"].select(range(100))):
    if prediction["label"]=="LABEL_0" and datapoint["label"]==1:
        print(datapoint["question"])
        print(datapoint["context"])
        print()

What type of hypersensitivity is associated with allergies?
Type IV reactions are involved in many autoimmune and infectious diseases, but may also involve contact dermatitis (poison ivy).

Who first described dynamic equilibrium?
Simple experiments showed that Galileo's understanding of the equivalence of constant velocity and rest were correct.

